In [ ]:
#!pip install https://git.ligo.org/cds/gpstime/-/archive/0.4.4/gpstime-0.4.4.zip

In [2]:
import json
import os
import folium
import geopandas as gpd
from pyproj import CRS, Transformer
import datetime
import gpstime
import requests

# Querying ATL08 Entwine Point Tiles

This notebook provides sample code for querying and visualizing the Entwine Point Tiles Store for ATL08 V003.

[Entwine Point Tiles (EPT)](https://entwine.io/entwine-point-tile.html) are a new "cloud-optimized" storage format for large point cloud data.

## Variables

The data columns in the EPT data correspond to the following ATL08 variables:

* Z - `gtx/land_segments/dem_h`
* ElevationLow - `gtx/land_segments/terrain/h_te_best_fit`
* HeightAboveGround - `gtx/land_segments/canopy/h_canopy`
* OriginId - `OriginId` is a numeric reference to an origin file, and references for all `OriginId`s can be found in `ept-sources/list.json`. The origin file is a las file passed as `origin` to the pdal EPT reader.
* GpsTime - `GpsTime` is the addition of the value of the HDF field `/ancillary_data/atlas_sdp_gps_epoch` (`1198800018`) and `gtx/land_segments/delta_time`.

Read more about those variables in the [ATL08 Data Dictionary](https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL08_data_dict_v003.pdf).

## Querying ATL08 EPT using the Features API

We can use the Features Service for some basic querying of the EPT Store.

> OGC API Features provides API building blocks to create, modify and query features on the Web.

[Documentation: OGC API - Features](https://www.ogc.org/standards/ogcapi-features)

### Query By Bounding Box

Note: The features endpoint will be deployed as part of the MAAP API and maap-py python library so this endpoint will change but documentation will update to reflect any new features.

In [9]:
%%time
# Format a request to the API
api_url = "https://obnrh8ozt0.execute-api.us-east-2.amazonaws.com/collections/Global/items"

# Make a request for a bounding box over Peru
bbox="-77,-26,300,-73,0,500"
limit = 1000
payload = {
    "f": "json",
    "limit": limit,
    "bbox": bbox
}

r = requests.get(api_url, params = payload)
# Get the results directly into a Geo Data Frame (saving to file not required but recommended)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.shape
adf.head()

CPU times: user 67.7 ms, sys: 980 µs, total: 68.6 ms
Wall time: 6 s


,geometry,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,...,GpsTime,Red,Green,Blue,ScanChannel,ClassFlags,ElevationLow,HeightAboveGround,OffsetTime,OriginId
0,POINT Z (-75.58000 -8.80000 310.06000),-75.58,-8.80,310.06,0.0,1.0,1.0,0.0,0.0,0.0,...,1.240313e+09,10.0,95.0,101.0,0.0,0.0,289.127,42.708,41512918.0,29028.0
1,POINT Z (-74.17000 -11.57000 471.59000),-74.17,-11.57,471.59,0.0,1.0,1.0,0.0,0.0,0.0,...,1.245357e+09,27.0,107.0,111.0,0.0,0.0,488.210,25.316,46556779.0,37945.0
2,POINT Z (-73.48000 -10.88000 329.13000),-73.48,-10.88,329.13,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258205e+09,25.0,106.0,110.0,0.0,0.0,314.425,35.354,59404688.0,58028.0
3,POINT Z (-74.16000 -10.18000 304.93000),-74.16,-10.18,304.93,0.0,1.0,1.0,0.0,0.0,0.0,...,1.258550e+09,12.0,96.0,101.0,0.0,0.0,302.224,20.358,59749777.0,58629.0
4,POINT Z (-73.51000 -9.49000 319.50000),-73.51,-9.49,319.50,0.0,1.0,1.0,0.0,0.0,0.0,...,1.266051e+09,17.0,100.0,105.0,0.0,0.0,296.021,27.648,67251450.0,71742.0


In [10]:
m = folium.Map(
    location=[adf.centroid[0].y, adf.centroid[0].x],
    zoom_start=10,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    adf,
    name = "geojson"
).add_to(m)

m

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


### Query by Granule Id

The UWG requested being able to query by granule id.

In [11]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
payload = {
    "f": "json",
    "origin": granule_id,
}

r = requests.get(api_url, params = payload)
api_geojson = r.json()
api_geojson.keys()
adf = gpd.GeoDataFrame.from_features(api_geojson["features"], crs='epsg:4326')
adf.head()

,geometry,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,...,GpsTime,Red,Green,Blue,ScanChannel,ClassFlags,ElevationLow,HeightAboveGround,OffsetTime,OriginId
0,POINT Z (-4.89000 11.43000 384.35000),-4.89,11.43,384.35,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,230.0,201.0,0.0,0.0,378.868,10.194,24724588.0,24.0
1,POINT Z (-4.97000 11.26000 402.93000),-4.97,11.26,402.93,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,221.0,191.0,0.0,0.0,401.890,8.355,24724590.0,24.0
2,POINT Z (-4.96000 11.09000 496.87000),-4.96,11.09,496.87,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,160.0,121.0,0.0,0.0,496.675,9.610,24724593.0,24.0
3,POINT Z (-4.97000 10.92000 559.00000),-4.97,10.92,559.00,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,255.0,117.0,73.0,0.0,0.0,557.005,11.124,24724595.0,24.0
4,POINT Z (-4.97000 10.66000 342.99000),-4.97,10.66,342.99,0.0,1.0,1.0,0.0,0.0,0.0,...,1.223525e+09,251.0,242.0,217.0,0.0,0.0,344.449,6.784,24724600.0,24.0


## PDAL Pipelines

PDAL pipelines can be used for a large set of data querying functionality, such as filtering data variables to be within a given range. Read more about different PDAL pipeline options here: https://pdal.io/pipeline.html.

### Requirements

To use PDAL, make sure to build a workspace using the "MAAP with PDAL" stack.

![Select MAAP with PDAL in Che](select-maap-with-pdal.png)

In [ ]:
## Define the global EPT source for our data queries
ept_store = "s3://cumulus-map-internal/file-staging/nasa-map/ATL08_ARD-beta___001/global/ept/ept.json"

## Define a function for running a pdal pipeline and returning the filename as output
def run_pipeline(pipeline_def, output_file):
    pipeline_def[-1]["filename"] = output_file
    pipeline_json =  json.dumps(pipeline_def)
    pipeline = pdal.Pipeline(pipeline_json)

    # remove the output file before building a new one
    if os.path.exists(output_file):
        os.remove(output_file)

    count = pipeline.execute()
    data = open(output_file, 'r').read()
    data = data.replace("[,{", "[{").replace("}{", "},{")
    open(output_file, 'w').write(data)
    return output_file

## Query by Granule using OriginId

To query for data from a particular granule, we can use the filename prefix to query the EPT store by `origin`.

In [ ]:
granule_id = "ATL08_20181026111023_04250106_003_01"
output_geojson = f"{granule_id}-subset.geojson"

pipeline_def = [
    {
        "type": "readers.ept",
        "filename": ept_store,
        "origin": f"{granule_id}.las"
    },
    {
        "type": "filters.reprojection",
        "out_srs":"EPSG:4326"
    },
    {
        "type" : "writers.text",
        "format": "geojson",
        "write_header": True
    }
]

run_pipeline(pipeline_def, output_geojson)

## Query by Bounding Box

To query for data for a bounding box, we first have to convert coordinates to the CRS of the EPT store (EPSG:3857).

In [ ]:
xmin, xmax = 10.1299,10.13
ymin, ymax = -0.0001,0
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
xmin, ymax = transformer.transform(xmin, ymax)
xmax, ymin = transformer.transform(xmax, ymin)
pdal_aoi_bounds = f"([{xmin}, {xmax}], [{ymin}, {ymax}])"
pdal_aoi_bounds

In [ ]:
pipeline_def = [
    {
        "type": "readers.ept",
        "filename": ept_store
    },
    {
        "type":"filters.crop",
        "bounds": pdal_aoi_bounds
    },
    {
        "type" : "writers.text",
        "format": "geojson",
        "write_header": True
    }
]

run_pipeline(pipeline_def, "spatial-subset.geojson")

## Convert GpsTime to Datetime

As described above, the data contains the field `GpsTime`:

>GPS Time (GPST) is a continuous time scale (no leap seconds) defined by the GPS Control segment on the basis of a set of atomic clocks at the Monitor Stations and onboard the satellites. It starts at 0h UTC (midnight) of January 5th to 6th 1980.

[Source gssc.esa.int](https://gssc.esa.int/navipedia/index.php/Time_References_in_GNSS#:~:text=GPS%20Time%20(GPST)%20is%20a,5th%20to%206th%201980%20(6.&text=0).)

In [ ]:
timestamp = gpstime.gps2unix(1224587557.969)
datetime.datetime.fromtimestamp(timestamp)

## Working with the results

Below are a number of examples of how to work with the geojson output from the example pipelines above.

### Reading into a data frame

In [ ]:
df = gpd.read_file(output_geojson)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
gdf

## Plotting

Finally, we can plot the data values

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

fig = plt.figure()
ax = plt.axes()
x = range(gdf.shape[0])

plt.plot(x, gdf.HeightAboveGround.astype(float), color='blue')

fig = plt.figure()
ax = plt.axes()
#plt.plot(x, gdf.ElevationLow.astype(float), color='red')
plt.plot(x, gdf.Z.astype(float), color='green')